In [1]:
import backtrader as bt
import pandas as pd
import os
import matplotlib
matplotlib.use('Agg')  # Use a non-GUI backend
import matplotlib.pyplot as plt
import backtrader.plot

# Patch Backtrader to use the correct pyplot reference
backtrader.plot.Plot_OldSync.mpyplot = plt


# ------------------------------
# Custom Data Feed with Composite Score
# ------------------------------
class FactorData(bt.feeds.PandasData):
    lines = ('composite_score',)
    params = (
        ('datetime', 0),
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', 1),
        ('volume', -1),
        ('openinterest', -1),
        ('composite_score', -1),
    )

In [2]:
# ------------------------------
# Factor-Based Long/Short Strategy
# ------------------------------
class FactorStrategy(bt.Strategy):
    params = dict(
        rebalance_days=5,
        top_n=2,
        bottom_n=2,
    )

    def __init__(self):
        self.counter = 0

    def next(self):
        self.counter += 1

        if self.counter % self.p.rebalance_days != 0:
            return

        # Gather composite scores from all data feeds
        scores = {}
        for d in self.datas:
            # Ensure we have enough data and composite score is valid
            if len(d) == 0 or not hasattr(d, 'composite_score') or d.composite_score[0] is None:
                continue
            try:
                score = float(d.composite_score[0])
                scores[d._name] = score
            except (ValueError, TypeError):
                continue

        print(f"[Day {self.counter}] Composite scores: {scores}")

        if not scores:
            print("No valid scores found. Skipping rebalance.")
            return

        top = sorted(scores, key=scores.get, reverse=True)[:self.p.top_n]
        bottom = sorted(scores, key=scores.get)[:self.p.bottom_n]

        print(f"Selected top: {top}")
        print(f"Selected bottom: {bottom}")

        # Close all current positions
        for d in self.datas:
            pos = self.getposition(d)
            if pos.size != 0:
                print(f"Closing position for {d._name}")
                self.close(d)

        # Go long on top-N
        for name in top:
            d = [data for data in self.datas if data._name == name][0]
            print(f"Going LONG on {name}")
            self.buy(d)

        # Go short on bottom-N
        for name in bottom:
            d = [data for data in self.datas if data._name == name][0]
            print(f"Going SHORT on {name}")
            self.sell(d)

In [3]:
# ------------------------------
# Backtrader Engine Setup
# ------------------------------
cerebro = bt.Cerebro()
cerebro.addstrategy(FactorStrategy)
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(commission=0.001)

In [4]:
# ------------------------------
# Load factor CSVs as Backtrader Data
# ------------------------------
import os
import pandas as pd

data_folder = "../data/factor_csvs"  # adjust if needed
files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

for file in files:
    df = pd.read_csv(os.path.join(data_folder, file))

    # Safe and consistent datetime parsing
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df.dropna(subset=['Date'], inplace=True)

    df.set_index('Date', inplace=True)

    # Create and add Backtrader data feed
    data = FactorData(dataname=df, name=file.replace('.csv', ''))
    cerebro.adddata(data)

In [5]:
# ------------------------------
# Run & Plot Results
# ------------------------------
print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
results = cerebro.run()
print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())

# Try plotting
figs = cerebro.plot(style='candlestick', iplot=False)

# Check if plot was successfully created
if figs and isinstance(figs[0], list) and hasattr(figs[0][0], 'savefig'):
    figs[0][0].savefig("backtest_plot.png", dpi=300)
    print("Plot saved to backtest_plot.png")
else:
    print("Plotting failed or no figure was returned. Skipping save.")

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00
Plotting failed or no figure was returned. Skipping save.
